### Testing and development for catchment and lake weight calclations ###
This was needed after modifying the way the catchment weights and metadata were caluclated,
improving it. Also, I had to fix a bug in the weighting function. So I updated the lake functions when I created the catchment functions also.

In [1]:
import ECCO_functions_v2 as ECCO

In [2]:
import pandas as pd
import osgeo.ogr
import sys, time, os, json, glob
import numpy as np
import pyproj
import h5py
import csv 
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
def Lake_surfaceweights_meta(nc_path,sbar=False):
    '''
    Purpose:          
    This program Generates metadata files used to speed up the final runs.
    This was forthe lakes (not catchments), and created a metadata text
    file, to be used as a mini-database in pandas.
    This is the second version if of a program to generate lake surface 
    weights and metadata. This was made after the program to do the same
    for the catchments. This creates hdf5 weight file, and .csv metadata.
    '''
        
    clim_dat,rlat,rlon,time,metadata,txtfname = ECCO.Read_CORDEX_V2(nc_path)
    vname, m1, m2, dexp, m3, m4, m5, m6, drange_orignial = metadata 
 
    var_type = clim_dat.standard_name       # What kind of CORDEX data?
    dat_loaded = clim_dat[0,:,:]            # Load CORDEX data into RAM
    print np.shape(dat_loaded)
    rlat_loaded = rlat[:]
    rlon_loaded = rlon[:]

    lake_file = 'Lakes/ecco-biwa_lakes_v.0.2.shp'
    
    thefilename = 'lake_weights'
    FILE= 'Lakes/Weights/' + thefilename +'.h5'                # Set up HDF5 file output
    if os.path.isfile(FILE):
        #print 'HDF5 File already exists. Leaving loop so you dont clobber it by accident.'
        #print 'To run this function, decide manually if you want to remove it or not.'
        #return
        print 'hdf weights file exists, removing it...'
        os.remove(FILE)
    else:
        print 'Creating file: ',FILE
    fweights = h5py.File(FILE,'w')
    
    # set and write header info for the metadata file
    metacsv = 'Lakes/Metadata/Lake_meta.csv'
    if os.path.isfile(metacsv) == True:
        print 'Earlier metadata exists. Erasing it...'
        os.remove(metacsv)
    tmplist = ['num','EB_id','area','npix','ypix','xpix'] 
    ECCO.write_metadata_csv(mfname=metacsv,meta_list=tmplist)
    
    #orog = ECCO.Height_CORDEX()
    ShapeData = osgeo.ogr.Open(lake_file)
    TheLayer = ShapeData.GetLayer(iLayer=0)
    dolakes=range(TheLayer.GetFeatureCount())

    if sbar:
        icnt = 0
    
    for num in dolakes[0:1000]:
        feature1 = TheLayer.GetFeature(num) 
        lake_feature = feature1.ExportToJson(as_object=True)
        lake_cart = ECCO.Path_LkIsl_ShpFile(lake_feature['geometry']['coordinates']) 
        EB_id = lake_feature['properties']['EBhex'][2:]
        lake_altitude=lake_feature['properties']['vfp_mean']

        lake_rprj = ECCO.Path_Reproj(lake_cart,False)

        sub_clim,sub_rlat,sub_rlon = ECCO.TrimToLake(lake_rprj,dat_loaded,rlat_loaded,
                                                        rlon_loaded,off = 3, show = False) 
        weight_mask = ECCO.Pixel_Weights(lake_rprj,sub_clim,sub_rlat,sub_rlon)


        pix_truth = (weight_mask > 0.0)      # Count how many 
        pxnum = len(weight_mask[pix_truth])  #  pixels of data are needed.
    
        ypix = -99
        xpix = -99
        if pxnum == 1:
            xxx,yyy = ECCO.Get_LatLonLim(lake_rprj.vertices)  # Find upp./low.lake lims.
            ypix = (ECCO.Closest(rlat,yyy[0]))                # For lakes of one pixel  
            xpix = (ECCO.Closest(rlon,xxx[0]))
        if pxnum < 1:
            pxnum = 1  # Small bug where it thinks lakes dont exist, no biggy...
        if pxnum > 1:
            ECCO.Write_HDF_weights(fw=fweights,EB_id=EB_id,weights=weight_mask) 
        
        tmpmeta =[num,EB_id, ECCO.Area_Lake_and_Islands(lake_cart),pxnum,ypix,xpix]
        ECCO.write_metadata_csv(mfname=metacsv,meta_list=tmpmeta)
        
        #print num,EB_id, ECCO.Area_Lake_and_Islands(lake_cart),pxnum,ypix,xpix
        if sbar:
            icnt=icnt+1
            if (float(icnt) % 10.) == 0.0:
                ECCO.Update_Progress(float(icnt)/len(dolakes))
    fweights.close()
    return 

In [37]:
#ncfile= '/uio/kant/geo-metos-u1/blaken/datadisk/ECCO/CORDEX/Data_CORDEX/tas_EUR-11_ICHEC-EC-EARTH_historical_r1i1p1_KNMI-RACMO22E_v1_day_19660101-19701231.nc'
ncfile ='CORDEX/tas_EUR-11_ICHEC-EC-EARTH_rcp45_r1i1p1_KNMI-RACMO22E_v1_day_20960101-21001231.nc'

Lake_surfaceweights_meta(nc_path=ncfile,sbar=True)

(412, 424)
hdf weights file exists, removing it...
Earlier metadata exists. Erasing it...
Progress: [----------------------------------------] 0% 

##Now use the test metadata to create a new routine for main processing##

In [3]:
lake_meta = pd.read_csv('Lakes/Metadata/Lake_meta.csv')
lake_meta = lake_meta.set_index('EB_id')
lake_meta.index[1]

'ccc59c'

In [5]:
#lake_meta

In [29]:
# Example of reading the data from a hdf5 file after creation using the metdatada as a lookup
#filein = 'Catchments/Weights/catchment_weights.h5'
#f = h5py.File(filein,"r")
# Going to test the weighting for the lakes (.npy) files, and also for the catchment hdf5 file.
# will read them in a loop, and sum the pixels. Will stop if the weight masks are not equal to
# 1 at any time.

acnt = 0
for n in lake_meta.index:
    if lake_meta.npix[n] > 1:
        with h5py.File('Lakes/Weights/lake_weights.h5','r') as fp:
            tmparray = fp[n]
            tmparray = tmparray[:,:]
        if tmparray.sum() > 1.01:
            acnt += 1.
            print n, tmparray.sum()
print 'Found problems in', acnt,' lakes'

# Found problems in 2026.0  lakes: too little weight
# Found problems in 2072.0  lakes: too much weight
# After change: Found problems in 0  lakes

599f21 1.01356
854aec 1.01046
1ccbad 1.02334
a96b25 1.01708
60bfab 1.01972
Found problems in 5.0  lakes


In [40]:
import time as clock

In [41]:
def Fast_v4(nc_path, lake_file, outputprefix,lstart=0,lstop=275265,
                       hexlist=None,tt=None,plots = False,rprt=False,sbar=False,
                       rprt_loop=False):
    '''
    Input:
    
           hexlist      If not None, this must be a list of hexcodes which match lake codes.
                        These codes will be the list of lakes to be processed, with
                        (regardless of lstart / lstop settings). The list should be ascii values
                        in any order. E.g. hexlist = ['a2204','155980','d23e4a','7aa917']
    
     k.w.agrs:   
    
           plots        True or False(default). If True, preview plots are created. I reccomend
                        using this feature carefully. Unless you have set a very small number of
                        lakes, this will produce a huge number of plots!
    
           rprt         Returns information on how long the program took to load the data,
                        complete, and how many lakes were processed.
        
           rprt_loop    Returns info on how long each specific lake took (can be a lot...)
    
           sbar         Create a status bar, to show the progression through a large loop. Not
                        shown by default. As, when this pro is on MPI it is not a good feature.
    
     Notes:       Out of 275265 total lakes, 264532 of them are within one pixel of EUR-11 data.
                  Metadata called from lake hexcode as index: not shapefile feature number.
    '''
    if rprt:
        atime = clock.time()

    #lk_processed_inf = pd.read_csv('Metadata/Meta_Lakes.csv')  # Load metadata
    lake_meta = pd.read_csv('Lakes/Metadata/Lake_meta.csv') # Load the metadata
    lake_meta = lake_meta.set_index('EB_id')                # Index with EB_id
    
    ShapeData = osgeo.ogr.Open(lake_file)                  # Make a link to Lake Shape Files
    TheLayer = ShapeData.GetLayer(iLayer=0)
    
    clim_dat,rlat,rlon,timeCDX,metadata,txtfname = ECCO.Read_CORDEX_V2(nc_path) # CORDEX Read
    vname, m1, m2, dexp, m3, m4, m5, m6, drange_orignial = metadata     # Metadata from fname
    var_type = clim_dat.standard_name                              # What kind of CORDEX data?
    dat_loaded = clim_dat[:,:,:]                                   # Load CORDEX data
    rlat_loaded = rlat[:]
    rlon_loaded = rlon[:]
    
    orog = ECCO.Height_CORDEX()                                 # EUR-11 surface height data 
    
    if hexlist == None:                   # Set up the list of lakes to process:
        dolakes=np.arange(lstart,lstop,1) #If no Hexcodes, use lstart/lstop to form a list
    else:
        dolakes= lake_meta.num[test]      #Else gen. list of nums from hex code meta
    
    thefilename = 'Lakes_'+str.split(nc_path,'/')[-1][:-3]

    FILE= outputprefix + thefilename +'.h5'       # Set up HDF5 file output
    if os.path.isfile(FILE):
        print 'Earlier file already exists: Overwriting...'
        os.remove(FILE)
    else:
        print 'No file found. Creating: ',FILE
    f = h5py.File(FILE,'w')

    # LOOP OVER ALL LAKES (or specified lakes from lstart to lstop)
    if rprt:
        btime = clock.time()
    if sbar:
        icnt = 0
    for n in dolakes[27:33]:
        tlist = []
        feature1 = TheLayer.GetFeature(n)           # Get individ. lake in shapefile
        lake_feature = feature1.ExportToJson(as_object=True)
        lake_cart = ECCO.Path_LkIsl_ShpFile(lake_feature['geometry']['coordinates'])
        lake_altitude=lake_feature['properties']['stf_mean']
        EB_id = lake_feature['properties']['EBhex']
        EB_id = EB_id[2:]                           # Strip off the hexcode label 0x
        lake_rprj = ECCO.Path_Reproj(lake_cart,False)    # Reproj. lake to CORDEX plr. rotated
        if EB_id != lake_meta.index[n]:      # Some handy error check
            print 'Warning! Lake feature and metadata miss-match for some reason. Check it out:'
            print 'Problem at:',num,lake_meta.num[n],EB_id,lake_meta.index[n]
        if plots:     
            ECCO.Preview_Lake(lake_cart)        
            print 'Area in km^2 (not inc. islands):', ECCO.Area_Lake_and_Islands(lake_cart),         
            print ', No. xy bound. points:',len(lake_cart.vertices)
        if lake_meta.npix[EB_id] == 1:         # ONE PIXEL LAKES <<<
            ypix = lake_meta.ypix[EB_id]       # Get the pre-calc. pixel indexes...
            xpix = lake_meta.xpix[EB_id]       # ...calc in MT_Gen_SWeights() earlier
            if lake_altitude == None:                 # Some lakes don't have alitude values
                offset = -999.
            else:
                offset = ECCO.OnePix_HOffset(lake_altitude,orog[ypix, xpix],var_type)
            tlist = dat_loaded[:, ypix, xpix]
            if plots:
                plt.plot(tlist)
        else:         # LAKES OF MORE THAN ONE PIXEL 
            with h5py.File('Lakes/Weights/lake_weights.h5','r') as fp:
                tmparray = fp[EB_id]
                weight_mask = tmparray[:,:]      # Load the precalculated weight data
            
            sub_clim,sub_rlat,sub_rlon = ECCO.TrimToLake3D(lake_rprj,dat_loaded,rlat_loaded,
                                                      rlon_loaded,
                                                      off = 3, show = False)
            
            if ((var_type == 'air_temperature')| (var_type == 'surface_air_pressure')): 
                sub_orog,sub_rlat,sub_rlon = ECCO.TrimToLake(lake_rprj,orog,rlat_loaded,
                                                            rlon_loaded,off = 3, show = False)
                if lake_altitude == None:       # Some lakes don't have alitude values
                    offset = -999.
                else:
                    hght,offset = ECCO.Orographic_Adjustment(weight_mask,sub_orog,
                                                        lake_altitude,clim_dat,chatty=False)
            else:
                hght = -999.         # If no altitude data then 
                offset = -999.       # set height and offset to missing vals
            
            
            tlist = ECCO.Weighted_Mean_3D(weight_mask, sub_clim, chatty=False)  # Here's the t-series
            
            if plots:
                ECCO.Show_LakeAndData(lake_rprj,dat_loaded[0,:,:],rlat,rlon,zoom=3.)
                ECCO.Preview_Weights(lake_rprj,weight_mask,sub_rlat,sub_rlon)
                plt.plot(tlist)
                
        if rprt_loop:
            print '\rStats:',(float(n)/float(lstop))*100.,'% ',n,EB_id,offset,lake_altitude
        if sbar:
            icnt=icnt+1
            if (float(icnt) % 10.) == 0.0:
                ECCO.Update_Progress(float(icnt)/float(len(dolakes)-1))

        ECCO.Write_HDF(f,EB_id,tlist,offset,lake_meta.area[n])  # Write inside function
        feature1=0
        lake_feature = 0

    f.close()                                # Close the HDF5 file after the lake loop finishes
    #subprocess.call(["gzip", FILE])          # Compress the file and remove original with gzip             
    if rprt:
            print '\nTime to read data: %4.2f sec'%(btime - atime)
            print 'Time to Process %i lakes: %4.2f sec'%(len(dolakes),clock.time() - btime)
    return

In [3]:
ncfile = 'CORDEX/tas_EUR-11_ICHEC-EC-EARTH_rcp45_r1i1p1_KNMI-RACMO22E_v1_day_20960101-21001231.nc'

lkfile = 'Lakes/ecco-biwa_lakes_v.0.2.shp'

ECCO.Fast_v4(nc_path=ncfile, lake_file=lkfile, outputprefix='',lstart=0,lstop=10,
             plots = False,rprt=True,sbar=True,rprt_loop=False)

No file found. Creating:  Lakes_tas_EUR-11_ICHEC-EC-EARTH_rcp45_r1i1p1_KNMI-RACMO22E_v1_day_20960101-21001231.h5
Progress: [########################################] 100% 

Time to read data: 29.19 sec
Time to Process 10 lakes: 0.21 sec


In [4]:
lake_meta = pd.read_csv('Lakes/Metadata/Lake_meta.csv')
lake_meta = lake_meta.set_index('EB_id')

In [20]:
# Example of reading the data from a hdf5 file after creation using the metdatada as a lookup
#filein = 'Catchments/Weights/catchment_weights.h5'
#f = h5py.File(filein,"r")
# Going to test the weighting for the lakes (.npy) files, and also for the catchment hdf5 file.
# will read them in a loop, and sum the pixels. Will stop if the weight masks are not equal to
# 1 at any time.

acnt = 0
for n in lake_meta.index[200:250]:
    if lake_meta.npix[n] > 1:
        with h5py.File('Lakes/Weights/lake_weights.h5','r') as fp:
            tmparray = fp[n]
            tmparray = tmparray[:,:]
            if tmparray.sum() != 1.:
                print n, tmparray.sum(),
                #pix_weights /= pix_weights.sum()
                tmparray /= tmparray.sum()
                print tmparray.sum()
                #plt.imshow(tmparray,interpolation='none',cmap=plt.cm.gray)
                #plt.show()

42f085 0.998918 1.0
43f298 0.995141 1.0
c78c76 0.999786 1.0
f73adf 0.998658 1.0
c8d0ed 0.998443 1.0


In [39]:
with h5py.File('Lakes/Weights/lake_weights.h5','r') as fp:
    tmparray = fp['c8d0ed']
    tmparray = tmparray[:,:]
    tmparray /= tmparray.sum()

In [36]:
atest = tmparray != 0.
tmparray[atest]
np.sum(tmparray[atest])

0.99844325

In [33]:
t1.sum()

1.0